In [1]:
#Importing all the required Libraries

import pandas as pd
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
from tensorflow.contrib import rnn
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

# Code Snippet remove the duplicates from the intrusion detection dataset
print ("Data Preprocessing Started.....\n")
traindataframe = pd.read_csv("C:/Users/manp/Documents/IoT/Final/ApplicationLayer/ApplicationLayerTrainingData1.txt",header = None,engine = 'python',sep=",")
recordcount = len(traindataframe)
print ("Original number of records in the training dataset before removing duplicates is: " , recordcount)
traindataframe.drop_duplicates(subset=None, inplace=True)  # Python command to drop duplicates
newrecordcount = len(traindataframe)
print ("Number of records in the training dataset after removing the duplicates is :", newrecordcount,"\n")
#Dropping the labels to a different dataset which is used to train the recurrent neural network classifier
train_X = traindataframe.drop(traindataframe.columns[41],axis=1)
labels = traindataframe.drop(traindataframe.columns[0:41],axis=1)


# Convert Categorial data to the numerical data for the efficient classification
train_X[train_X.columns[1:4]] = train_X[train_X.columns[1:4]].stack().rank(method='dense').unstack()

#labels.replace(['normal.',!'normal.'],[1,0],inplace = True)
labels[labels[41]!='normal.'] = 0
labels[labels[41]=='normal.'] = 1
#print (labels[41].value_counts())

inputX = train_X.loc[:,train_X.columns[0:41]].astype(float)

labels.columns = ["y1"]
#labels.loc[:,('y2')] = [0,1,1,1,0,1,0,0,0,]

labels.loc[:,('y2')]=labels['y1'] ==0
labels.loc[:,('y2')] = labels['y2'].astype(int)
inputY = labels.as_matrix()
inputX = inputX.as_matrix()


# Code Snippet for test data
print ("Data Preprocessing Started.....\n")
testdataframe = pd.read_csv("C:/Users/manp/Documents/IoT/Final/ApplicationLayer/ApplicationLayerTestData1.txt",header = None,engine = 'python',sep=",")
testrecordcount = len(testdataframe)
print ("Original number of records in the testing dataset before removing duplicates is: " , testrecordcount)
testdataframe.drop_duplicates(subset=None, inplace=True)  # Python command to drop duplicates
newtestrecordcount = len(testdataframe)
print ("Number of records in the training dataset after removing the duplicates is :", newtestrecordcount,"\n")
#Dropping the labels to a different dataset which is used to train the recurrent neural network classifier
test_X = testdataframe.drop(testdataframe.columns[41],axis=1)
testlabels = testdataframe.drop(testdataframe.columns[0:41],axis=1)


# Convert Categorial data to the numerical data for the efficient classification
test_X[test_X.columns[1:4]] = test_X[test_X.columns[1:4]].stack().rank(method='dense').unstack()

#labels.replace(['normal.',!'normal.'],[1,0],inplace = True)
testlabels[testlabels[41]!='normal'] = 0
testlabels[testlabels[41]=='normal'] = 1
#print (testlabels[41].value_counts())

inputX_test = test_X.loc[:,test_X.columns[0:41]].astype(float)

testlabels.columns = ["y1"]

testlabels.loc[:,('y2')]=testlabels['y1'] ==0
testlabels.loc[:,('y2')] = testlabels['y2'].astype(int)
inputY_test = testlabels.as_matrix()
inputX_test = inputX_test.as_matrix()



# Hyper Parameters
learning_rate = 0.001
training_epochs = 10
display_step =1
num_layers = 1
#Input Placeholders
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,shape = [None,41], name = "x-input")
    y = tf.placeholder(tf.float32, shape = [None,2],name = "y-input")
#Weights and Biases
with tf.name_scope("weights"):
    W = tf.Variable(tf.random_normal([41,2]))

with tf.name_scope("biases"):
    b = tf.Variable(tf.random_normal([2]))

#Model
with tf.name_scope("splitx"):
    newx = tf.split(x,1,0)
with tf.name_scope("multicellconfig"):
    multicell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.GRUCell(41) for _ in range (num_layers)], state_is_tuple=True)
    
with tf.variable_scope('mygrucell'):
    outputs,states = tf.contrib.rnn.static_rnn(multicell,newx,dtype=tf.float32, scope = None)

with tf.name_scope("output"):
    output = tf.add(tf.matmul(outputs[-1],W),b)

with tf.name_scope("cross_entropy"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = output))
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(y,1))
    cast = tf.cast(correct_prediction, tf.float32)
    accuracy = tf.reduce_mean(cast)
#create summary for the cost and accuracy
tf.summary.scalar("cost",cost)
tf.summary.scalar("accuracy", accuracy)
summary_op = tf.summary.merge_all()
logs_path = "tmp/gru/test10"
with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        writer = tf.summary.FileWriter(logs_path, graph = tf.get_default_graph())
        for i in range (training_epochs):
            _,summary = sess.run([optimizer,summary_op], feed_dict = {x:inputX, y:inputY})
            writer.add_summary(summary,i)
    
            if (i) % display_step == 0:
                print (i,"Cost for this epoch is",sess.run(cost, feed_dict ={x :inputX,y:inputY}))
        print ("Accuracy",accuracy.eval(feed_dict = {x:inputX_test,y:inputY_test}))
        print ("test Output is :", sess.run(output,feed_dict = {x:inputX_test, y:inputY_test}))
        print ("test labels are :", sess.run(y,feed_dict = {x:inputX_test, y:inputY_test}))
        pred_class = sess.run(tf.argmax(output,1),feed_dict = {x:inputX_test,y:inputY_test})
        labels_class = sess.run(tf.argmax(y,1),feed_dict = {x:inputX_test,y:inputY_test})
        conf = tf.contrib.metrics.confusion_matrix(labels_class,pred_class,dtype = tf.int32)
        print ("confusion matrix \n", sess.run(conf, feed_dict={x:inputX_test, y:inputY_test}))
        n = tf.cast(labels_class,tf.int64)
        newaccuracy = tf.contrib.metrics.accuracy(pred_class,n)
        print ("new accuracy", sess.run (newaccuracy, feed_dict = {x:inputX_test, y:inputY_test}))
        TP = conf[0,0]
        FN = conf [0,1]
        FP = conf[1,0]
        TN = conf[1,1]
        AccConf = (TP+TN)/(TP+FP+TN+FN)
        print ("Accuracy calculated through confusion matrix", sess.run (AccConf, feed_dict = {x:inputX_test,y:inputY_test}))
        # Precision
        Precision = TP/(TP+FP)
        print ("Precision \n",sess.run(Precision,feed_dict ={x:inputX_test, y:inputY_test}))
        #Recall
        Recall = TP/(TP+FN)
        print ("Recall (DR)\n", sess.run(Recall,feed_dict={x:inputX_test,y:inputY_test}))
        #F score
        FScore = 2*((Precision*Recall)/(Precision+Recall))
        print ("F1 Score is \n",sess.run(FScore,{x:inputX_test, y:inputY_test}))
        #False Alarm Rate
        FAR = FP/(FP+TN)
        print ("False Alarm Rate is \n",sess.run(FAR,feed_dict ={x:inputX_test,y:inputY_test}))
        #Efficiency
        Efficiency = Recall/FAR
        print("Efficincy is \n",sess.run(Efficiency,feed_dict = {x:inputX_test, y:inputY_test}))




Data Preprocessing Started.....

Original number of records in the training dataset before removing duplicates is:  7000
Number of records in the training dataset after removing the duplicates is : 7000 

Data Preprocessing Started.....

Original number of records in the testing dataset before removing duplicates is:  6200
Number of records in the training dataset after removing the duplicates is : 6200 

0 Cost for this epoch is 5.81112
1 Cost for this epoch is 5.44382
2 Cost for this epoch is 5.18188
3 Cost for this epoch is 4.92499
4 Cost for this epoch is 4.68146
5 Cost for this epoch is 4.44261
6 Cost for this epoch is 4.21419
7 Cost for this epoch is 4.00923
8 Cost for this epoch is 3.80604
9 Cost for this epoch is 3.60479
Accuracy 0.632258
test Output is : [[  0.2971884   -1.05368376]
 [ -2.20881653   3.1637845 ]
 [  3.97210217  -0.27501708]
 ..., 
 [  1.28061664   6.72052193]
 [  2.19289112  -2.11706829]
 [  0.83300078  10.16396809]]
test labels are : [[ 0.  1.]
 [ 0.  1.]
 [ 0